In [1]:
import pandas as pd
pd.set_option('display.max_rows', None) # Print all rows

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df[df['Lugar de residencia'] == 'Huelva']

,Lugar de residencia,Medida,Valor
0,Huelva,Población,513170
1,Huelva,Confirmados PDIA,33435
2,Huelva,Confirmados PDIA 14 días,918
3,Huelva,Tasa PDIA 14 días,"178,88808776818598"
4,Huelva,Confirmados PDIA 7 días,415
5,Huelva,Total Confirmados,33620
6,Huelva,Curados,31666
7,Huelva,Fallecidos,390


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  33435.0
<ipython-input-7-31b732756b19>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  9148.0
<ipython-input-8-e81bfb40e710>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

<ipython-input-9-f09dd2050cc1>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
<ipython-input-9-f09dd2050cc1>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
<ipython-input-9-f09dd2050cc1>:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]


In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 1262 personas en los últimos 7 días 

Un positivo PCR cada 475 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,33435.0,415.0,918.0,513170.0,80.869887,178.888088,153.0
Huelva-Costa,19266.0,242.0,513.0,289548.0,83.578543,177.172697,104.0
Huelva (capital),9148.0,114.0,303.0,143837.0,79.256380,210.655117,46.0
Condado-Campiña,10873.0,147.0,329.0,156231.0,94.091442,210.585607,40.0
Ayamonte,1358.0,24.0,27.0,21104.0,113.722517,127.937832,17.0
Moguer,1415.0,34.0,64.0,21867.0,155.485435,292.678465,9.0
Sierra de Huelva-Andévalo Central,2942.0,25.0,73.0,67391.0,37.096942,108.323070,8.0
Isla Cristina,2195.0,16.0,30.0,21393.0,74.790819,140.232786,8.0
Aljaraque,1179.0,18.0,27.0,21474.0,83.822297,125.733445,8.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() != 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
San Juan del Puerto,593.0,24.0,41.0,9411.0,255.020720,435.660397,7.0
Cerro de Andévalo (El),156.0,5.0,9.0,2327.0,214.868930,386.764074,2.0
Valverde del Camino,665.0,8.0,38.0,12750.0,62.745098,298.039216,1.0
Moguer,1415.0,34.0,64.0,21867.0,155.485435,292.678465,9.0
Manzanilla,109.0,5.0,6.0,2118.0,236.071766,283.286119,2.0
Palos de la Frontera,731.0,24.0,33.0,11742.0,204.394481,281.042412,2.0
Bonares,304.0,11.0,16.0,6060.0,181.518152,264.026403,5.0
Paymogo,78.0,3.0,3.0,1162.0,258.175559,258.175559,2.0
Alosno,188.0,9.0,10.0,3933.0,228.832952,254.258835,6.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Valverde del Camino,665.0,8.0,38.0,12750.0,62.745098,298.039216,1.0,0.210526
Almonte,1919.0,14.0,54.0,24507.0,57.126535,220.345207,3.0,0.259259
Bollullos Par del Condado,1038.0,7.0,22.0,14387.0,48.655036,152.915827,5.0,0.318182
Almonaster la Real,70.0,1.0,3.0,1788.0,55.928412,167.785235,0.0,0.333333
Villablanca,192.0,1.0,3.0,2885.0,34.662045,103.986135,0.0,0.333333
Trigueros,334.0,3.0,9.0,7862.0,38.158229,114.474688,0.0,0.333333
Sierra de Huelva-Andévalo Central,2942.0,25.0,73.0,67391.0,37.096942,108.323070,8.0,0.342466
Huelva (capital),9148.0,114.0,303.0,143837.0,79.256380,210.655117,46.0,0.376238
Palma del Condado (La),1197.0,9.0,23.0,10801.0,83.325618,212.943246,2.0,0.391304
